# Superstore PySpark Assignment
This notebook answers Questions 9–14 from the Module 1 Assignment using PySpark and the Superstore dataset.

In [44]:
# Install Spark and Java (only needed on Colab)
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

tar: spark-3.4.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max as spark_max, count, expr
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import expr

# Initialize Spark
spark = SparkSession.builder.appName("SuperstoreAssignment").getOrCreate()

In [50]:
# Replace with your actual raw GitHub CSV URL
csv_url = "https://raw.githubusercontent.com/SamuelBoye/assignments/refs/heads/main/Superstore-2.csv"

# Download the file
!wget -O Superstore.csv $csv_url

# Read into Spark
df = spark.read.csv("Superstore.csv", header=True, inferSchema=True)

# Cast numeric fields
df = df.withColumn("Sales", expr("try_cast(Sales as double)"))
df = df.withColumn("Profit", expr("try_cast(Profit as double)"))

# Preview data
df.show(5)


--2025-06-02 20:00:58--  https://raw.githubusercontent.com/SamuelBoye/assignments/refs/heads/main/Superstore-2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2167101 (2.1M) [text/plain]
Saving to: ‘Superstore.csv’

Superstore.csv      100%[===================>]   2.07M  --.-KB/s    in 0.07s   

2025-06-02 20:00:59 (28.0 MB/s) - ‘Superstore.csv’ saved [2167101/2167101]

+--------------------+-----------+---------+-----------+--------------------+----------------+---------------+--------+-------+--------------------+-----------------+----------+--------+--------------+--------------+-----------+-------------------+--------+------+----------+------+----------+--------------+-------------+--------------+----------+
|            Category|       City|Containe

### Q9: Who is the customer that generated the most sales?

In [12]:
top_customer = df.groupBy("Customer Name").sum("Sales").orderBy("sum(Sales)", ascending=False)
top_customer.show(1)

+------------------+----------+
|     Customer Name|sum(Sales)|
+------------------+----------+
|Joshua N. Milligan|  123748.0|
+------------------+----------+
only showing top 1 row



### Q10: Highest average sales per transaction for any city

In [13]:
avg_sales = df.groupBy("City").agg(avg("Sales").alias("avg_sales")).orderBy("avg_sales", ascending=False)
avg_sales.show(1)

+----------+---------+
|      City|avg_sales|
+----------+---------+
|Barrington|   7029.0|
+----------+---------+
only showing top 1 row



### Q11: Highest total profit for any item

In [19]:
# Q11: Total profit per item, then find the highest
from pyspark.sql.functions import sum as spark_sum

df.groupBy("Item") \
  .agg(spark_sum("Profit").alias("total_profit")) \
  .orderBy("total_profit", ascending=False) \
  .show(1)


+------------+------------+
|        Item|total_profit|
+------------+------------+
|Imation 3.5"|   1207461.0|
+------------+------------+
only showing top 1 row



### Q12: Largest number of transactions for any combination of state and customer segment

In [54]:
from pyspark.sql.functions import countDistinct

df.groupBy("State", "Customer Segment") \
  .agg(countDistinct("Order ID").alias("unique_transactions")) \
  .orderBy("unique_transactions", ascending=False) \
  .show(1)

+-----+----------------+-------------------+
|State|Customer Segment|unique_transactions|
+-----+----------------+-------------------+
|Texas|       Corporate|                246|
+-----+----------------+-------------------+
only showing top 1 row



### Q13: Highest average profit for any city-state combination

In [53]:
from pyspark.sql.functions import col

# Filter out known corrupted state values
df_clean = df.filter(~col("State").isin("Regular Air", "Express Air", "Delivery Truck", "Second Class", "First Class"))

# Register new clean temp view
df_clean.createOrReplaceTempView("superstore")

# Now rerun Q13 query
spark.sql("""
SELECT City, State, ROUND(AVG(Profit), 2) AS avg_profit
FROM superstore
WHERE City IS NOT NULL AND State IS NOT NULL AND Profit IS NOT NULL
GROUP BY City, State
ORDER BY avg_profit DESC
LIMIT 1
""").show()



+------------+-------+----------+
|        City|  State|avg_profit|
+------------+-------+----------+
|Douglasville|Georgia|    5379.0|
+------------+-------+----------+



In [51]:
df.filter(df.City == "Troutdale").select("City", "State", "Ship Mode", "Profit").show()


+---------+-----------+--------------+-------+
|     City|      State|     Ship Mode| Profit|
+---------+-----------+--------------+-------+
|Troutdale|     Oregon|   Regular Air| -773.0|
|Troutdale|Regular Air|            10|97060.0|
|Troutdale|     Oregon|   Regular Air|    1.0|
|Troutdale|     Oregon|   Regular Air|  990.0|
|Troutdale|     Oregon|Delivery Truck| 4749.0|
+---------+-----------+--------------+-------+

